In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
ratings = pd.read_csv('ratings.csv')
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [ ]:
movies = pd.read_csv('movies.csv')
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [ ]:
merged = pd.merge(movies[['movieId']], ratings[['userId','movieId', 'rating']], on='movieId')
pivot = merged.pivot(index='userId', columns='movieId', values='rating')
pivot=pivot.fillna(0)
pivot

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Convert to NumPy array
array = pivot.values
array

array([[4. , 0. , 4. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [2.5, 2. , 2. , ..., 0. , 0. , 0. ],
       [3. , 0. , 0. , ..., 0. , 0. , 0. ],
       [5. , 0. , 0. , ..., 0. , 0. , 0. ]])

## Collaborative Multiarm bandits

In [ ]:
# Define a function to select the next movie to play based on the current values and exploration parameter
def select_movie(exploration_param, num_movies, movie_ratings, movie_values,user_id):
    if np.random.random() < exploration_param:
        # Choose a random movie to play
        return np.random.randint(num_movies)
    else:
        # Choose the movie with the highest value among the unrated movies
        unrated_movies = np.where(movie_ratings[user_id] == 0)[0]
        return np.argmax(movie_values[unrated_movies])

In [ ]:
def bandit_collaborative_filtering(movie_ratings,user_id):

    np.random.seed(user_id)

    # Initialize the bandit algorithm with a random initial value for each movie
    num_users, num_movies = movie_ratings.shape
    movie_values = np.random.normal(size=num_movies)

    # Define the exploration parameter
    exploration_param = 0.1

    # Define the number of plays for each movie
    num_plays = np.zeros(num_movies)

    # Define the total reward for each movie
    total_reward = np.zeros(num_movies)
        # Define the number of iterations to run the bandit algorithm
    num_iterations = 1000

    # Run the bandit algorithm for the specified number of iterations
    for i in range(num_iterations):
        # Select a movie to play
        movie_to_play = select_movie(exploration_param, num_movies, movie_ratings, movie_values,user_id)

        # Update the number of plays for the selected movie
        num_plays[movie_to_play] += 1

        # Calculate the reward for the selected movie based on the movie ratings dataset and the user's ratings
        reward = movie_ratings[user_id, movie_to_play]

        # Update the total reward for the selected movie
        total_reward[movie_to_play] += reward

        # Update the value of the selected movie based on the updated total reward and number of plays
        movie_values[movie_to_play] = total_reward[movie_to_play] / num_plays[movie_to_play]

    # Recommend the top 10 movies based on the final values, excluding already rated movies
    rated_movies = np.where(movie_ratings[user_id] != 0)[0]
    top_movies = np.argsort(movie_values)[-10:]
    return movie_values,[movie for movie in top_movies if movie not in rated_movies]

# Content Base Filtering

In [ ]:
genres = np.unique(np.concatenate(movies['genres'].str.split('|').values))
genres

array(['(no genres listed)', 'Action', 'Adventure', 'Animation',
       'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
       'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance',
       'Sci-Fi', 'Thriller', 'War', 'Western'], dtype='<U18')

In [ ]:
def user_profile(active_user,movies,ratings,genres):
    user_profile=ratings.loc[active_user,['movieId','rating']]
    user_item_profile=pd.DataFrame(columns=genres, index=user_profile['movieId'])
    for i, row in movies.loc[user_profile['movieId'].values].iterrows():
        genres = row['genres'].split('|')
        user_item_profile.loc[i, tuple(genres)] = 1
    user_item_profile=user_item_profile.fillna(0)
    user_item_profile = user_item_profile.mul(user_profile['rating'].values, axis=0)
    user_item_profile=user_item_profile.sum()
    return user_item_profile / user_item_profile.sum()


In [ ]:
def similarity(active_user,movies,ratings,genres,user_item_profile):
    item_profile=pd.DataFrame(columns=genres, index=movies.index)
    for i, row in movies.iterrows():
        genres = row['genres'].split('|')
        item_profile.loc[i, tuple(genres)] = 1
    item_profile=item_profile.fillna(0)
    # print(item_profile)
    mapped_item_profile=item_profile.mul(user_item_profile.values.T)
    # print(mapped_item_profile)
    return pd.DataFrame(cosine_similarity(user_item_profile.values.T.reshape(1, -1), mapped_item_profile.values), columns=mapped_item_profile.index, index=[active_user])
    #

In [ ]:
def recommendation(active_user,movies,ratings,genres,bandit_movies):
    user_item_profile=user_profile(active_user,movies,ratings,genres)
    # print(user_item_profile)
    similarties=similarity(active_user,bandit_movies,ratings,genres,user_item_profile)
    similarties=similarties.loc[:, similarties.iloc[0].sort_values(ascending=False).index]
    # recommending 3 movies
    similarties=similarties.iloc[:,:3]
    return movies.loc[similarties.columns,'title']


## Main Driving Code

In [ ]:
active_user=int(input("Enter The User Id "))
movie_values,recommended_movies=bandit_collaborative_filtering(array,active_user)
print("Using Collaborative Multiarm bandits the 10 recommended movies for User", active_user)
for movie in recommended_movies:
    print(f"Movie {movie}: Value = {movie_values[movie]}")

In [ ]:
movies=movies.set_index('movieId')
ratings=ratings.set_index('userId')

In [ ]:
print("Top 3 movies that will be Recommended",recommendation(active_user,movies,ratings,genres,movies.iloc[recommended_movies]))